# I will introduce some tips.


# Information
* [OpenCV Python Tutorial](https://docs.opencv.org/master/d6/d00/tutorial_py_root.html)  
* [OpenCV API Umage Processing](https://docs.opencv.org/3.0-beta/modules/imgproc/doc/imgproc.html)

# Preparation

In [ ]:
import os
import sys
import random
from random import randint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add,UpSampling2D,Concatenate
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#

In [ ]:
# Set some parameters
im_width = 101
im_height = 101
im_chan = 1
basicpath = '../input/'
path_train = basicpath + 'train/'
path_test = basicpath + 'test/'

path_train_images = path_train + 'images/'
path_train_masks = path_train + 'masks/'
path_test_images = path_test + 'images/'

In [ ]:
# Loading of training/testing ids and depths

train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

len(train_df)

In [ ]:
train_df["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]
train_df["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]
train_df["coverage"] = train_df.masks.map(np.sum) / pow(101, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i

x_train_sorted_by_coverage=np.array(train_df.sort_values(by='coverage').images.tolist()).reshape(-1, 101, 101, 1)
x_train_sorted_by_z=np.array(train_df.sort_values(by='z').images.tolist()).reshape(-1, 101, 101, 1)
y_train_sorted_by_coverage=np.array(train_df.sort_values(by='coverage').masks.tolist()).reshape(-1, 101, 101, 1)
y_train_sorted_by_z=np.array(train_df.sort_values(by='z').masks.tolist()).reshape(-1, 101, 101, 1)
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.tolist()).reshape(-1, 101, 101, 1), 
    np.array(train_df.masks.tolist()).reshape(-1, 101, 101, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state= 1234)

# How to display images on the screen using OpenCV

In [ ]:
import cv2
from IPython.display import display, Image
def cvshow(image, format='.png', rate=255 ):
    decoded_bytes = cv2.imencode(format, image*rate)[1].tobytes()
    display(Image(data=decoded_bytes))
    return

#usage
cvshow(x_train_sorted_by_coverage[0])

# A method of arranging image arrays in a tile shape

In [ ]:
def imgtile(imgs,tile_w):
    assert imgs.shape[0]%tile_w==0,"'imgs' cannot divide by 'th'."
    r=imgs.reshape((-1,tile_w)+imgs.shape[1:])
    return np.hstack(np.hstack(r))

#usage
tiled = imgtile(x_train_sorted_by_coverage[:100],10)
cvshow(tiled)
tiled.shape

# How to display while switching images using the slide bar

In [ ]:
from ipywidgets import interact,IntSlider
@interact
def f(i=IntSlider(min=0,max=39,step=1,value=0)):
    cvshow(imgtile(x_train_sorted_by_coverage[i*100:(i+1)*100],10))

# How to display all training data as thumbnails

In [ ]:
all=imgtile(x_train_sorted_by_z,80)
cvshow(cv2.resize( all, (80*16,50*16), interpolation=cv2.INTER_LINEAR ))

In [ ]:
all=imgtile(y_train_sorted_by_z,80)
cvshow(cv2.resize( all, (80*16,50*16), interpolation=cv2.INTER_LINEAR ))

In [ ]:
all=imgtile(x_train_sorted_by_coverage,80)
cvshow(cv2.resize( all, (80*16,50*16), interpolation=cv2.INTER_LINEAR ))

In [ ]:
all_train=imgtile(y_train_sorted_by_coverage,80)
cvshow(cv2.resize( all_train, (80*16,50*16), interpolation=cv2.INTER_LINEAR ))

# How to pad four sides of an image into a mirror image

In [ ]:
def create_padded(imgs,dst_hw,odd_mirror=True):
    imgs_hw=imgs.shape[1:3]
    assert((np.array(imgs_hw)<=np.array(dst_hw)).all())
    #calc
    pad_t, pad_l = int((dst_hw[0]-imgs_hw[0])/2), int((dst_hw[1]-imgs_hw[1])/2)
    pad_b, pad_r = dst_hw[0]-imgs_hw[0]-pad_t, dst_hw[1]-imgs_hw[1]-pad_l
    #copy
    ret=np.zeros((imgs.shape[0],)+dst_hw+(imgs.shape[3],))
    ret[:,pad_t:pad_t+imgs_hw[0],pad_l:pad_l+imgs_hw[1],:]=imgs[:,:,:,:]
    #pad
    ofs=1 if odd_mirror else 0
    ret[:,:pad_t,:,:]=ret[:,pad_t*2+ofs:pad_t+ofs:-1,:,:]
    ret[:,-pad_b:,:,:]=ret[:,-pad_b-1-ofs:-pad_b*2-1-ofs:-1,:,:]
    ret[:,:,:pad_l,:]=ret[:,:,pad_l*2+ofs:pad_l+ofs:-1,:]
    ret[:,:,-pad_r:,:]=ret[:,:,-pad_r-1-ofs:-pad_r*2-1-ofs:-1,:]    
    return ret

#usage
x_train_padded=create_padded(x_train_sorted_by_coverage,(192,192))
y_train_padded=create_padded(y_train_sorted_by_coverage,(192,192))
cvshow(imgtile(x_train_padded[:40],10))

# How to randomly cut out images using Keras's callback function
(I have not learned how to use the generator function)


In [ ]:
def rand_crop(x,crop_hw):
    x_hw=x.shape[1:3]
    assert((np.array(x_hw)>=np.array(crop_hw)).all())
    crop_ys_max, crop_xs_max = np.array(x_hw)-np.array(crop_hw)
    ret=[]
    for xx in x:
        ys, xs = randint(0,crop_ys_max), randint(0,crop_xs_max)
        ret.append(xx[ys:ys+crop_hw[0],xs:xs+crop_hw[1],:])
    return np.array(ret)

from keras.callbacks import Callback
class RandCrop(Callback):
    def __init__(self,src,dst):
        self.src=src
        self.dst=dst
    def on_epoch_end(self, epoch, logs=None):
        self.dst=rand_crop(self.src,self.dst.shape[1:3])

In [ ]:
#usage

#Definie UNet Builder
def conv_block_mod(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n
def level_block_mod(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block_mod(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block_mod(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)#再帰
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block_mod(n, dim, acti, bn, res)
    else:
        m = conv_block_mod(m, dim, acti, bn, res, do)
    return m
def UNet_mod(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
         dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
    i = Input(shape=img_shape)
    o = level_block_mod(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)#Unet
    o = Conv2D(out_ch, 1, activation='sigmoid')(o)
    return Model(inputs=i, outputs=o)

#Create UNet
model=UNet_mod((None,None,1),start_ch=16,depth=5,batchnorm=True,dropout=0.5)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#Set Callback
xy_train_padded=np.concatenate((x_train_padded,y_train_padded),axis=3)
xy_train_cropped=xy_train_padded[:,:64,:64,:]
randcrop = RandCrop(xy_train_padded,xy_train_cropped)

#Train
history = model.fit(xy_train_cropped[...,0:1], xy_train_cropped[...,1:2],
                    epochs=1,
                    batch_size=64,
                    callbacks=[randcrop])

# Check x,y,pred

In [ ]:
def cvshow_BGR(image, format='.png', rate=255 ):
    decoded_bytes = cv2.imencode(format, image*rate)[1].tobytes()
    display(Image(data=decoded_bytes))
    return

In [ ]:
def create_xyp(x,y,p,p_thresh=.5):
    if p_thresh>0:
        p=(p>p_thresh).astype(np.float32)
    xyp=np.concatenate((x/2+.25,y/8+.5,-p/8+.5),axis=-1)
    return np.array( [cv2.cvtColor((i*255).astype(np.uint8),cv2.COLOR_YCrCb2BGR) for i in xyp] )

In [ ]:
#usage
x = xy_train_cropped[...,0:1]
y = xy_train_cropped[...,1:2]
p = model.predict(x)
cvshow_BGR(imgtile( create_xyp(x,y,p,.4)[2000:2100],10))